The main program that uses the database and defined python scripts to run the analysis

In [1]:
#This non-descript folder contains all the standalone python modules
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

In [2]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('data/seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('data/qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('data/driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('data/constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('data/enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [3]:
scores = []
scores = list(map(runSimulation, range(40), repeat(seasonsData), repeat(qualiResultsData), repeat(driversData), 
                    repeat(constructorsData), repeat(enginesData)))
print(np.median(scores))

0.5671794871794871


So, now when the data has been analysed, it's time to use it to predict the next qualification's results! In this simplified version, the grand prix is specified by the parameter circuit.

In [4]:
(drivers, constructors, engines) = runSimReturnDrivConstEng(seasonsData, qualiResultsData, 
                                                            driversData, constructorsData, enginesData)

In [5]:
newDrivers = json.load(open('data/newDrivers.json'))["drivers"]
newDrivers = {int(did): cid for did, cid in newDrivers.items()}

#print(newDrivers)
driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = drivers[int(did)].constructor.name
with open('../F1PredictWeb/driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

# Here we can set the circuit to simulate. This will also help generate the correct properties for updating the server.

In [6]:
circuit = 32
circuitName = "Mexican"

with open('../F1PredictWeb/circuitName.txt', 'w') as fp:
    fp.write(circuitName);

In [7]:
driverResults = {} # {did: {position: amount}}
for did, cid in newDrivers.items():
    driverResults[int(did)] = {}
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers, drivers, constructors, engines)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(drivers[int(did)].name, drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
with open('../F1PredictWeb/predictions.json', 'w') as fp:
    json.dump(driverResults, fp)

#for i, res in enumerate(scoreList):
    #print("{0}: {1} ({2})".format(i+1, drivers[res[0]].name, drivers[res[0]].constructor.name))        

Lewis Hamilton (Mercedes):
	1: 57.4 %
	2: 41.6 %
	3: 0.4 %
	4: 0.3 %
	10: 0.1 %
	15: 0.2 %
Fernando Alonso (McLaren):
	16: 0.2 %
	17: 5.4 %
	18: 32.0 %
	19: 56.9 %
	20: 5.5 %
Kimi Räikkönen (Ferrari):
	2: 1.5 %
	3: 44.0 %
	4: 53.9 %
	10: 0.1 %
	15: 0.1 %
	20: 0.4 %
Sebastian Vettel (Ferrari):
	1: 0.1 %
	2: 0.9 %
	3: 54.7 %
	4: 43.5 %
	10: 0.3 %
	15: 0.1 %
	20: 0.4 %
Romain Grosjean (Haas F1 Team):
	5: 0.8 %
	6: 5.3 %
	7: 17.4 %
	8: 20.4 %
	9: 15.2 %
	10: 14.4 %
	11: 10.5 %
	12: 6.7 %
	13: 6.5 %
	14: 2.2 %
	15: 0.4 %
	16: 0.2 %
Nico Hülkenberg (Renault):
	5: 0.1 %
	6: 0.6 %
	7: 3.0 %
	8: 10.0 %
	9: 15.5 %
	10: 16.5 %
	11: 17.3 %
	12: 15.4 %
	13: 12.5 %
	14: 6.8 %
	15: 2.1 %
	20: 0.2 %
Sergio Pérez (Force India):
	5: 7.4 %
	6: 23.0 %
	7: 31.8 %
	8: 14.4 %
	9: 9.1 %
	10: 4.7 %
	11: 4.7 %
	12: 2.9 %
	13: 1.3 %
	14: 0.4 %
	15: 0.2 %
	20: 0.1 %
Daniel Ricciardo (Red Bull):
	5: 0.3 %
	6: 4.4 %
	7: 16.9 %
	8: 24.1 %
	9: 17.0 %
	10: 11.7 %
	11: 9.4 %
	12: 8.4 %
	13: 5.0 %
	14: 2.1 %
	15: 0.5 %


In [ ]:
predictedEntrants = []

for did, cid in newDrivers.items():
    entry = [
        drivers[did].pwr, 
        drivers[did].constructor.pwr, 
        drivers[did].constructor.engine.pwr,
        drivers[did].trackpwr[circuit],
        drivers[did].constructor.trackpwr[circuit],
        drivers[did].constructor.engine.trackpwr[circuit]
    ]
    